# Weeks 7 & 8 Exercises

## So Much Data Candy, Seriously

Amelia Farrell

DSC540

4/27/23


In [1]:

# Connecting to google drive
from google.colab import drive 
drive.mount('/content/drive')

# Change to the google drive directory
%cd /content/drive/MyDrive/Colab\ Notebooks/DSC540

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/DSC540


In [2]:
# importing libraries needed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Step 1: Understand the Data

For this anaylsis, we want to better understand candy prefrences by age. 
To start, we need to see the data we are working with and removing any missing "AGE" values.

In [3]:
# Loading the flat file into a Pandas DataFrame
df = pd.read_excel("candyhierarchy2017.xlsx", sheet_name='responses')

In [4]:
# view header names
header_names = df.columns
print(header_names)

Index(['Internal ID', 'Q1: GOING OUT?', 'Q2: GENDER', 'Q3: AGE', 'Q4: COUNTRY',
       'Q5: STATE, PROVINCE, COUNTY, ETC', 'Q6 | 100 Grand Bar',
       'Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)',
       'Q6 | Any full-sized candy bar', 'Q6 | Black Jacks',
       ...
       'Q8: DESPAIR OTHER', 'Q9: OTHER COMMENTS', 'Q10: DRESS', 'Unnamed: 113',
       'Q11: DAY', 'Q12: MEDIA [Daily Dish]', 'Q12: MEDIA [Science]',
       'Q12: MEDIA [ESPN]', 'Q12: MEDIA [Yahoo]', 'Click Coordinates (x, y)'],
      dtype='object', length=120)


In [5]:
# viewing the dataframe with headers and first rows
df.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


In [6]:
# checking for missing vaules
missing_values = df.isnull()
# Count the number of missing values in each column
num_missing = missing_values.sum()
# Print the result
print(num_missing)

Internal ID                    0
Q1: GOING OUT?               110
Q2: GENDER                    41
Q3: AGE                       84
Q4: COUNTRY                   64
                            ... 
Q12: MEDIA [Daily Dish]     2375
Q12: MEDIA [Science]        1098
Q12: MEDIA [ESPN]           2361
Q12: MEDIA [Yahoo]          2393
Click Coordinates (x, y)     855
Length: 120, dtype: int64


## Step 2: Remove Extra Columns

Chapter 7: Filter out missing data

Since we are only interested in understanding candy preferences by age, we want to remove the DRESS & MEDIA columns, as these are extra survey questions that we will not be interested in for this analysis. 

In [7]:
# removing ("dropping") inessential columns
df2 = df.drop(columns=['Q10: DRESS','Internal ID','Click Coordinates (x, y)','Unnamed: 113','Q1: GOING OUT?', 'Q4: COUNTRY', 'Q5: STATE, PROVINCE, COUNTY, ETC','Q9: OTHER COMMENTS','Q8: DESPAIR OTHER','Q7: JOY OTHER', 'Q12: MEDIA [Daily Dish]','Q12: MEDIA [Science]','Q12: MEDIA [ESPN]','Q12: MEDIA [Yahoo]'])

# view new header names
new_header_names = df2.columns
print(new_header_names)

Index(['Q2: GENDER', 'Q3: AGE', 'Q6 | 100 Grand Bar',
       'Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)',
       'Q6 | Any full-sized candy bar', 'Q6 | Black Jacks',
       'Q6 | Bonkers (the candy)', 'Q6 | Bonkers (the board game)',
       'Q6 | Bottle Caps', 'Q6 | Box'o'Raisins',
       ...
       'Q6 | Tolberone something or other', 'Q6 | Trail Mix', 'Q6 | Twix',
       'Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein',
       'Q6 | Vicodin', 'Q6 | Whatchamacallit Bars', 'Q6 | White Bread',
       'Q6 | Whole Wheat anything', 'Q6 | York Peppermint Patties',
       'Q11: DAY'],
      dtype='object', length=106)


## Step 3: Rename Headers

Chapter 7: Replace values & Manipulate Strings

We also need to rename the columns due to the Q##: leading each.

In [8]:
# replace the header names of select columns
df2.columns = df2.columns.str.replace('[1,:,|,Q,2,3,4,5,6,7,8,9,0, ,(,)]', '')

<ipython-input-8-d07b9a9e4763>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.columns = df2.columns.str.replace('[1,:,|,Q,2,3,4,5,6,7,8,9,0, ,(,)]', '')


In [9]:
# view new new header names
new_header_names2 = df2.columns
print(new_header_names2)

Index(['GENDER', 'AGE', 'GrandBar',
       'Anonymousbrownglobsthatcomeinblackandorangewrappers\ta.k.a.MaryJanes',
       'Anyfull-sizedcandybar', 'BlackJacks', 'Bonkersthecandy',
       'Bonkerstheboardgame', 'BottleCaps', 'Box'o'Raisins',
       ...
       'Tolberonesomethingorother', 'TrailMix', 'Twix',
       'Vialsofpurehighfructosecornsyrupformain-liningintoyourvein', 'Vicodin',
       'WhatchamacallitBars', 'WhiteBread', 'WholeWheatanything',
       'YorkPeppermintPatties', 'DAY'],
      dtype='object', length=106)


## Step 4: Removing/Filtering out missing values

As we saw in Step 1, there are 84 rows with missing ages. We will need to filter out those rows for our anaylsis. We are chooseing not to replace these vaules with the medium, due to the high number of missing values. Using the medium age for each of the 84 rows, could skew our data. 

In [10]:
# next we want remove any of the rows with missing ages
df3 = df2[df2.AGE.notnull()]

In [11]:
# checking for missing vaules after removing the 84 missing from Age
missing_values3 = df3.isnull()
# Count the number of missing values in each column
num_missing3 = missing_values3.sum()
# Print the result
print(num_missing3)

GENDER                                                                    7
AGE                                                                       0
GrandBar                                                                696
Anonymousbrownglobsthatcomeinblackandorangewrappers\ta.k.a.MaryJanes    683
Anyfull-sizedcandybar                                                   623
                                                                       ... 
WhatchamacallitBars                                                     769
WhiteBread                                                              706
WholeWheatanything                                                      695
YorkPeppermintPatties                                                   654
DAY                                                                     671
Length: 106, dtype: int64


In [12]:
# there are also text values in our AGE column that will need to be removed
# converting the Ages to numeric data type, ignoring non-numeric values
df3['AGE'] = pd.to_numeric(df3['AGE'], errors='coerce')
# drop rows with missing values (strings)
df3 = df3.dropna()
df3.head()

<ipython-input-12-04d9fce7cda4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['AGE'] = pd.to_numeric(df3['AGE'], errors='coerce')


,GENDER,AGE,GrandBar,Anonymousbrownglobsthatcomeinblackandorangewrappers\ta.k.a.MaryJanes,Anyfull-sizedcandybar,BlackJacks,Bonkersthecandy,Bonkerstheboardgame,BottleCaps,Box'o'Raisins,...,Tolberonesomethingorother,TrailMix,Twix,Vialsofpurehighfructosecornsyrupformain-liningintoyourvein,Vicodin,WhatchamacallitBars,WhiteBread,WholeWheatanything,YorkPeppermintPatties,DAY
1,Male,44.0,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,...,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Sunday
3,Male,40.0,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,...,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,Sunday
8,Male,40.0,JOY,DESPAIR,JOY,MEH,MEH,MEH,MEH,DESPAIR,...,JOY,DESPAIR,JOY,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,Sunday
13,Male,43.0,JOY,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,...,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,Sunday
14,Female,37.0,MEH,JOY,JOY,MEH,JOY,DESPAIR,MEH,DESPAIR,...,JOY,DESPAIR,JOY,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,JOY,Sunday


## Step 5: Count Occurrences

Charpter 10: Grouping with Functions

Charpter 8: Reshape

Next we want to count the number of time each candy was rated as "JOY" to see which was the most popular amoung the groups that said their age.

In [13]:
# Counting the occurrences of "JOY" in all columns by creating a function
string_to_count = 'JOY'
column_counts = {}

for column in df3.columns:
    count = df3[column].astype(str).str.count(string_to_count).sum()
    column_counts[column] = count

# Storing our count of "JOY" in a new DataFrame
counts_df3 = pd.DataFrame(column_counts.items(), columns=['Column', 'Count'])
print(counts_df3)

                                                Column  Count
0                                               GENDER      0
1                                                  AGE      0
2                                             GrandBar    506
3    Anonymousbrownglobsthatcomeinblackandorangewra...     92
4                                Anyfull-sizedcandybar    855
..                                                 ...    ...
101                                WhatchamacallitBars    503
102                                         WhiteBread     27
103                                 WholeWheatanything     56
104                              YorkPeppermintPatties    584
105                                                DAY      0

[106 rows x 2 columns]


In [14]:
# find max value
max_value = counts_df3['Count'].max()
filteredmax_df = counts_df3[counts_df3['Count'] == max_value]
print(filteredmax_df)

                  Column  Count
4  Anyfull-sizedcandybar    855


## Step 6: Combine and Merge Datasets

Charpter 8: Combine and Merge Datasets

Charpter 10: Split/Apply/Combine

Now that we know that Anyfull-sizedcandybar is the winner of the entire group. How do we know how age changes this? To find out, lest create 3 tables, one for each age group.

In [15]:
# one for folks between 20 and 35
# creating three new data sets based on age
filteredage_Young = df3[(df3['AGE'] >= 20)]
filteredage_Young2 = filteredage_Young[(filteredage_Young['AGE'] <= 35)]

for column in filteredage_Young2.columns:
    count = filteredage_Young2[column].astype(str).str.count(string_to_count).sum()
    column_counts[column] = count

# Storing our count of "JOY" in a new DataFrame
filteredage_Young_counts = pd.DataFrame(column_counts.items(), columns=['Column', 'Count'])
# adding a new column with age grouping
filteredage_Young_counts['age_group'] = 'young'

In [16]:
# one for folks between 35 and 50
# creating three new data sets based on age
filteredage_Mid = df3[(df3['AGE'] > 35)]
filteredage_Mid2 = filteredage_Mid[(filteredage_Mid['AGE'] <= 50)]

for column in filteredage_Mid2.columns:
    count = filteredage_Mid2[column].astype(str).str.count(string_to_count).sum()
    column_counts[column] = count

# Storing our count of "JOY" in a new DataFrame
filteredage_Mid_counts = pd.DataFrame(column_counts.items(), columns=['Column', 'Count'])
# adding a new column with age grouping
filteredage_Mid_counts['age_group'] = 'middle'

In [17]:
# one for folks over 50
# creating three new data sets based on age
filteredage_old = df3[(df3['AGE'] > 50)]

for column in filteredage_old.columns:
    count = filteredage_old[column].astype(str).str.count(string_to_count).sum()
    column_counts[column] = count

# Storing our count of "JOY" in a new DataFrame
filteredage_old_counts = pd.DataFrame(column_counts.items(), columns=['Column', 'Count'])
# adding a new column with age grouping
filteredage_old_counts['age_group'] = 'old'

In [39]:
# appending the three
df_combined = pd.concat([filteredage_Young_counts, filteredage_Mid_counts, filteredage_old_counts], ignore_index=True)
# filtering out Gender and Age
filteredgroup = df_combined[~df_combined['Column'].isin(['GENDER', 'AGE'])]
filteredgroup.head()

,Column,Count,age_group
2,GrandBar,104,young
3,Anonymousbrownglobsthatcomeinblackandorangewra...,15,young
4,Anyfull-sizedcandybar,212,young
5,BlackJacks,10,young
6,Bonkersthecandy,17,young


In [19]:
# so what is the top candy from each group?
# find the maximum value based on a grouping

# Young
young_group = filteredgroup.loc[filteredgroup['age_group'] == 'young']
max_value_young = young_group.loc[young_group['Count'] > 2, 'Count'].max()
filter_young_max = young_group.loc[young_group['Count'] == max_value_young]
print(filter_young_max)

                  Column  Count age_group
4  Anyfull-sizedcandybar    212     young


In [20]:
# Middle Age
mid_group = filteredgroup.loc[filteredgroup['age_group'] == 'middle']
max_value_mid = mid_group.loc[mid_group['Count'] > 2, 'Count'].max()
filter_mid_max = mid_group.loc[mid_group['Count'] == max_value_mid]
print(filter_mid_max)

                    Column  Count age_group
110  Anyfull-sizedcandybar    436    middle


In [21]:
# Old Age
old_group = filteredgroup.loc[filteredgroup['age_group'] == 'old']
max_value_old = old_group.loc[old_group['Count'] > 2, 'Count'].max()
filter_old_max = old_group.loc[old_group['Count'] == max_value_old]
print(filter_old_max)

                    Column  Count age_group
216  Anyfull-sizedcandybar    177       old


According to the output, the mijority like any full-sizedcandybar regardless of age group.

## Step 7: Pivot

 Chapter 8: Pivot the data

 Number of total "JOY"s from each age group.

In [22]:
# pivot the data by age group to see how many from each group marked "JOY" on all candy
pivot = pd.pivot_table(filteredgroup, values = 'Count', columns = 'age_group', aggfunc='sum', fill_value=0)
pivotdata = pd.DataFrame(pivot)
pivotdata

age_group,middle,old,young
Count,17512,7312,9370


## Step 8: Date & Time

Chapter 11: Generate date range

Chapter 11: Convert to periods and back

Since our data does not include dates and times, we will be generating a random date and time using the 'random' package.


In [41]:
# importing the random package
import random
from datetime import datetime, timedelta

# specifying the start and end dates we would like to use
start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 5, 1)

# adding a random date to every row
def add_date(row):
  row["date"]=(start_date + timedelta(days=random.randint(0, (end_date - start_date).days)))
  return row

datawithdate =filteredgroup.apply(add_date, axis=1)
datawithdate

,Column,Count,age_group,date
2,GrandBar,104,young,2023-01-24
3,Anonymousbrownglobsthatcomeinblackandorangewra...,15,young,2022-07-20
4,Anyfull-sizedcandybar,212,young,2022-10-10
5,BlackJacks,10,young,2023-02-11
6,Bonkersthecandy,17,young,2023-01-08
...,...,...,...,...
313,WhatchamacallitBars,77,old,2022-03-25
314,WhiteBread,5,old,2022-11-01
315,WholeWheatanything,7,old,2022-03-22
316,YorkPeppermintPatties,134,old,2022-05-29


In [42]:
# adding a column with the new dates converted to month (peirod)
datawithdate['month'] = pd.to_datetime(datawithdate['date']).dt.to_period('M')
datawithdate

,Column,Count,age_group,date,month
2,GrandBar,104,young,2023-01-24,2023-01
3,Anonymousbrownglobsthatcomeinblackandorangewra...,15,young,2022-07-20,2022-07
4,Anyfull-sizedcandybar,212,young,2022-10-10,2022-10
5,BlackJacks,10,young,2023-02-11,2023-02
6,Bonkersthecandy,17,young,2023-01-08,2023-01
...,...,...,...,...,...
313,WhatchamacallitBars,77,old,2022-03-25,2022-03
314,WhiteBread,5,old,2022-11-01,2022-11
315,WholeWheatanything,7,old,2022-03-22,2022-03
316,YorkPeppermintPatties,134,old,2022-05-29,2022-05


In [45]:
# converting the month 'period' back to date and time
datawithdate['month'] = datawithdate['month'].apply(lambda x: pd.Period(x, freq='M').to_timestamp())
datawithdate


,Column,Count,age_group,date,month
2,GrandBar,104,young,2023-01-24,2023-01-01
3,Anonymousbrownglobsthatcomeinblackandorangewra...,15,young,2022-07-20,2022-07-01
4,Anyfull-sizedcandybar,212,young,2022-10-10,2022-10-01
5,BlackJacks,10,young,2023-02-11,2023-02-01
6,Bonkersthecandy,17,young,2023-01-08,2023-01-01
...,...,...,...,...,...
313,WhatchamacallitBars,77,old,2022-03-25,2022-03-01
314,WhiteBread,5,old,2022-11-01,2022-11-01
315,WholeWheatanything,7,old,2022-03-22,2022-03-01
316,YorkPeppermintPatties,134,old,2022-05-29,2022-05-01
